In [2]:
import boto3
import os
import json

def get_secret(secret_name):
    client = boto3.client('secretsmanager', region_name='us-east-1')
    response = client.get_secret_value(SecretId=secret_name)
    secret = json.loads(response['SecretString'])
    return secret

def download_folder_from_s3(bucket_name, local_folder, s3_client):
    os.makedirs(local_folder, exist_ok=True)
    
    paginator = s3_client.get_paginator('list_objects_v2')
    for page in paginator.paginate(Bucket=bucket_name):
        for obj in page.get('Contents', []):
            s3_path = obj['Key']
            local_path = os.path.join(local_folder, s3_path)
            local_dir = os.path.dirname(local_path)
            os.makedirs(local_dir, exist_ok=True)
            try:
                s3_client.download_file(bucket_name, s3_path, local_path)
                print(f'Successfully downloaded s3://{bucket_name}/{s3_path} to {local_path}')
            except Exception as e:
                print(f'Failed to download s3://{bucket_name}/{s3_path} to {local_path}: {e}')

# Retrieve secrets
secret_name = "S3InputBucket-RAG"  # Replace with your secret name
secrets = get_secret(secret_name)

# Extract secrets
bucket_name = secrets['bucket_name']
role_arn = secrets['role_arn']

# Set the local folder path for downloading
local_folder_download = './test'

# Create an STS client
sts_client = boto3.client('sts')

# Assume the role
response = sts_client.assume_role(RoleArn=role_arn, RoleSessionName='AssumeRoleSession')
credentials = response['Credentials']

# Configure AWS SDK with temporary credentials
s3_client = boto3.client('s3',
                         aws_access_key_id=credentials['AccessKeyId'],
                         aws_secret_access_key=credentials['SecretAccessKey'],
                         aws_session_token=credentials['SessionToken'])

# Call the function to download the folder
download_folder_from_s3(bucket_name, local_folder_download, s3_client)


Successfully downloaded s3://rag-llm-docs/CVE-2021-47194.json to ./tests/CVE-2021-47194.json
Successfully downloaded s3://rag-llm-docs/CVE-2021-47198.json to ./tests/CVE-2021-47198.json
Successfully downloaded s3://rag-llm-docs/CVE-2021-47520.json to ./tests/CVE-2021-47520.json
Successfully downloaded s3://rag-llm-docs/CVE-2021-47521.json to ./tests/CVE-2021-47521.json
Successfully downloaded s3://rag-llm-docs/CVE-2021-47525.json to ./tests/CVE-2021-47525.json
Successfully downloaded s3://rag-llm-docs/CVE-2021-47541.json to ./tests/CVE-2021-47541.json
Successfully downloaded s3://rag-llm-docs/CVE-2021-47571.json to ./tests/CVE-2021-47571.json
Successfully downloaded s3://rag-llm-docs/CVE-2022-48655.json to ./tests/CVE-2022-48655.json
Successfully downloaded s3://rag-llm-docs/CVE-2022-48657.json to ./tests/CVE-2022-48657.json
Successfully downloaded s3://rag-llm-docs/CVE-2022-48658.json to ./tests/CVE-2022-48658.json
Successfully downloaded s3://rag-llm-docs/CVE-2022-48662.json to ./tes

KeyboardInterrupt: 